# Vitamins


## Extract

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import date
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [5]:
foods = ['ברוקולי','בננה','תרד','אבוקדו','שזיף מיובש']

In [6]:
def buttonGrab(Xpath,kind = 'button'):
    
    if kind=='button':
        return WebDriverWait(driver,5).until(expected_conditions.element_to_be_clickable((By.XPATH, Xpath)))
    
    return WebDriverWait(driver,5).until(expected_conditions.element_to_be_selected((By.XPATH, Xpath)))

In [7]:
searchBoxXpath = '//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input'
searchXpath ='//*[@id="tsf"]/div[2]/div[1]/div[3]/center/input[1]'

def searcher(food):
    
    searchBox = buttonGrab(searchBoxXpath)
    searchBox.clear()
    searchBox.send_keys(food+' ערך תזונתי')
        
    buttonGrab(searchXpath).click()
      
    return driver.current_url

In [8]:
def scrape(url):
    
    
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    soup = BeautifulSoup(webpage, "html.parser")


    table = soup.find('table')
    table_rows = table.find_all('tr')

    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        res.append(row)   
        
    return res 

In [9]:
path = r'C:\Users\97250\Desktop\studied\R ,python\chromedriver.exe'
driver = webdriver.Chrome(path)
address = "https://www.google.com/"
driver.get(address)

In [74]:
def vitaminsTransposer(vit):
    
    val = vit[-1][0]
    vit.append(val)
    vit.append('%')
    return vit

In [11]:
def foodAncestorsTransposer(value):
    
    x = [s for s in value if s.isdigit()]

    indexBegin = value.index(x[0])
    indexEnd   = value.index(x[-1])
    
    ##rows.append()
    
    return [value[:indexBegin-1],value[indexBegin:indexEnd+1],value[indexEnd+1:]]


In [13]:
def scrapeOrderer(table):
    
    foodAncestors = table[3:15]
    vitamins = table[15:]
    
    vitamins      = [vitaminsTransposer(i[0]) for i in vitamnis]
    foodAncestors = [foodAncestorsTransposer(i[0]) for i in foodAncestors]
       
    transposedTable = foodAncestors.concat(vitamins)
    
    return transposedTable

In [30]:
data=[]

for food in foods:
    
    url = searcher(food)
    
    try:
        res = scrape(url)
    except:
        pass
    
    transposedTable = scrapeOrderer(res)
        
    if len(data) == 0:
        data = pd.DataFrame(transposedTable,columns=['vitamin',food])
    
    else: 
        data.merge(pd.DataFrame(transposedTable,columns=['vitamin',food]),on='vitamin')
        
    driver.get(address)

In [15]:
data = scrape(searcher('בננה'))

In [43]:
for i in foodAncestors:
    print(i[0])

Total Fat 0.3 g
Saturated fat 0.1 g
Polyunsaturated fat 0.1 g
Monounsaturated fat 0 g
Cholesterol 0 mg
Sodium 1 mg
Potassium 358 mg
Total Carbohydrate 23 g
Dietary fiber 2.6 g
Sugar 12 g
Protein 1.1 g
Caffeine 0 mg


In [52]:
##foodAncestors = data[3:15]
vitamins = data[15:]

In [67]:
[vitaminsTransposer(i) for i in vitamins]

[['Vitamin A', '1%', '', '%'],
 ['Vitamin C', '14%', '', '%'],
 ['Calcium', '0%', '', '%'],
 ['Iron', '1%', '', '%'],
 ['Vitamin D', '0%', '', '%'],
 ['Vitamin B-6', '20%', '', '%'],
 ['Cobalamin', '0%', '', '%'],
 ['Magnesium', '6%', '', '%']]

In [73]:
vitamins

[['Vitamin A', '1%', '', '%'],
 ['Vitamin C', '14%', '', '%'],
 ['Calcium', '0%', '', '%'],
 ['Iron', '1%', '', '%'],
 ['Vitamin D', '0%', '', '%'],
 ['Vitamin B-6', '20%', '', '%'],
 ['Cobalamin', '0%', '', '%'],
 ['Magnesium', '6%', '', '%']]

In [50]:

rows = []

def convertor(value):
    
    x = [s for s in value if s.isdigit()]

    indexBegin = value.index(x[0])
    indexEnd   = value.index(x[-1])
    
    #rows.append()
    
    return [value[:indexBegin-1],value[indexBegin:indexEnd+1],value[indexEnd+1:]]



In [28]:
for i in data:
    print(i[0].split(','))

['כמות לכל 100 g']
['קלוריות (kcal) 88.7']
['']
['סה"כ שומן 0.3 g']
['שומן רווי 0.1 g']
['שומן רב בלתי רווי 0.1 g']
['שומן חד בלתי רווי 0 g']
['כולסטרול 0 mg']
['נתרן 1 mg']
['אשלגן 358 mg']
['סה"כ פחמימות 23 g']
['סיבים תזונתיים 2.6 g']
['סוכר 12 g']
['חלבון 1.1 g']
['קפאין 0 mg']
['ויטמין A']
['ויטמין C']
['סידן']
['ברזל']
['ויטמין D']
['ויטמין B6']
['ויטמין B12']
['מגנזיום']


In [51]:
data = [] 

for vit in res:
    

    data.append([vit[0].split('(')[0],vit[-2]])

In [52]:
def hasNumbers(inputString):
     return any(char.isdigit() for char in inputString)

In [53]:
data

[['Thiamine ', '100 mg'],
 ['Riboflavin ', '75 mg'],
 ['Niacin ', '100 mg•'],
 ['Vitamin B6 ', '100 mg'],
 ['Folate ', '400 mcg○'],
 ['Vitamin B12 ', '300 mcg'],
 ['Biotin', '1000 mcg'],
 ['Pantothenic acid ', '500 mg'],
 ['Calcium ', '90 mg'],
 ['Inositol', '100 mg'],
 ['PABA ', '50 mg']]

In [55]:
data2=[]

for i in data:
    data2.append([i[0],int(int(i[1].split()[0])/2)])

    
data2 = pd.DataFrame(data2,columns=['Vitamin','Amount Per Capsule']).set_index('Vitamin')